In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import torch
torch.set_printoptions(precision=10)

from pytorch_lightning import Trainer
from models.ranked_transformer import HsqcRankedTransformer

from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.oneD_dataset import OneDDataModule
from datasets.hsqc_folder_dataset import FolderDataModule

from datasets.optional_2d_folder_dataset import OptionalInputDataModule
import yaml
from pytorch_lightning.loggers import TensorBoardLogger
torch.set_float32_matmul_precision('medium')

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")


In [3]:
from datasets.dataset_utils import  fp_loader_configer
fp_loader_configer.select_version("Hash_Entropy")  #("MFP_Specific_Radius")
fp_loader = fp_loader_configer.fp_loader


choosing Hash_Entropy_FP_loader


Here we compare models' performance over 7 kinds of inputs


In [4]:
# load model 

# checkpoint_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/stable_argsort/flexible_models_best_FP/r0_r2_FP_trial_1/checkpoints/epoch=32-all_inputs.ckpt")
checkpoint_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/more_specialized_models/train_on_all_data_possible_with_jittering/no_mw_only_hsqc_trial_2/checkpoints/epoch=132-step=17290.ckpt")

model_path = checkpoint_path.parents[1]


hyperpaerameters_path = model_path / "hparams.yaml"

with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
    
FP_building_type = hparams['FP_building_type'].split("_")[-1]
# only_2d = not hparams['use_oneD_NMR_no_solvent']

radius = int(hparams['FP_choice'].split("_")[-1])
fp_loader.setup(out_dim=hparams['out_dim'], max_radius=radius)


del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
# hparams['use_peak_values'] = False




finish entropy list
Hash_Entropy_FP_loader is setup, out_dim=16384, max_radius=6


In [25]:
args = hparams
datamodule = FolderDataModule(f"/workspace/SMILES_dataset", FP_choice=args["FP_choice"], input_src=["HSQC", ], batch_size=2, parser_args=args, fp_loader=fp_loader,) 
# data_module = data_mux(parser, args["modelname"], args["datasrc"], args["FP_choice"], args["bs"], args["ds"], args)
trainer = Trainer( accelerator="auto", accumulate_grad_batches=args["accumulate_grad_batches_num"])
model = HsqcRankedTransformer.load_from_checkpoint(checkpoint_path, fp_loader=fp_loader, **hparams)

model.setup_ranker()


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/non_collision_FP_rankingset_max_radius_6_dim_16384_stacked_together/FP.pt


In [12]:
test_result = trainer.test(model, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


loaded full info indices





Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │      0.998291015625       │
│   test/mean_active_bits   │            2.0            │
│     test/mean_ce_loss     │   0.015541097149252892    │
│       test/mean_cos       │            0.0            │
│       test/mean_f1        │            0.0            │
│     test/mean_jaccard     │            0.0            │
│    test/mean_mean_rank    │         275518.0          │
│    test/mean_neg_loss     │  0.00023753655841574073   │
│    test/mean_pos_loss     │   0.015303561463952065    │
│  test/mean_pos_neg_loss   │   0.015541098080575466    │
│    test/mean_precision    │            0.0            │
│     test/mean_rank_1      │            0.0            │
│     test/mean_rank_10     │            0.0            │
│     test/mean_rank_5      │            0.0            │
│     test/mean_recall      │            0.0            │
└───────────────────────────┴───────────────────────────┘

In [26]:
model.setup_ranker()
model.logger_should_sync_dist = False

# my_logger.info(f"[Main] my process rank: {os.getpid()}")
trainer = Trainer( devices = 1, accumulate_grad_batches=args["accumulate_grad_batches_num"])

test_result = trainer.test(model, datamodule,ckpt_path=checkpoint_path)

loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/non_collision_FP_rankingset_max_radius_6_dim_16384_stacked_together/FP.pt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /root/gurusmart/MorganFP_prediction/reproduce_previous_works/more_specialized_models/train_on_all_data_possible_with_jittering/no_mw_only_hsqc_trial_2/checkpoints/epoch=132-step=17290.ckpt


loaded full info indices





LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /root/gurusmart/MorganFP_prediction/reproduce_previous_works/more_specialized_models/train_on_all_data_possible_with_jittering/no_mw_only_hsqc_trial_2/checkpoints/epoch=132-step=17290.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      Test metric                       ┃                      DataLoader 0                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                   test/mean_accuracy                   │                   0.9965030550956726                   │
│                 test/mean_active_bits                  │                   6.642751693725586                    │
│                   test/mean_ce_loss                    │                  0.042543765157461166                  │
│                     test/mean_cos                      │                  0.07873775064945221                   │
│                      test/mean_f1                      │                  0.05050705745816231                   │
│                   test/mean_jaccard                    │                  0.026904432103037834                  │
│                  test/mean_mean_rank                   │                      223103.0625                       │
│                   test/mean_neg_loss                   │                 0.0009390884661115706                  │
│                   test/mean_pos_loss                   │                  0.04160467907786369                   │
│                 test/mean_pos_neg_loss                 │                  0.042543765157461166                  │
│                  test/mean_precision                   │                   0.2185247838497162                   │
│                    test/mean_rank_1                    │                          0.0                           │
│                   test/mean_rank_10                    │                 0.00024654832668602467                 │
│                    test/mean_rank_5                    │                          0.0                           │
│                    test/mean_recall                    │                  0.028939727693796158                  │
│        test/rank_1_of_NP_class/Alkylresorcinols        │                          0.0                           │
│     test/rank_1_of_NP_class/Amino acid glycosides      │                          0.0                           │
│        test/rank_1_of_NP_class/Aminosugars and         │                          0.0                           │
│                    aminoglycosides                     │                                                        │
│   test/rank_1_of_NP_class/Anthranilic acid alkaloids   │                          0.0                           │
│         test/rank_1_of_NP_class/Apocarotenoids         │                          0.0                           │
│      test/rank_1_of_NP_class/Aromatic polyketides      │                          0.0                           │
│       test/rank_1_of_NP_class/Carotenoids (C40)        │                          0.0                           │
│       test/rank_1_of_NP_class/Carotenoids (C50)        │                          0.0                           │
│           test/rank_1_of_NP_class/Chromanes            │                          0.0                           │
│           test/rank_1_of_NP_class/Coumarins            │                          0.0                           │
│       test/rank_1_of_NP_class/Cyclic polyketides       │                          0.0                           │
│        test/rank_1_of_NP_class/Diarylheptanoids        │                          0.0                           │
│    test/rank_1_of_NP_class/Diazotetronic acids and     │                          0.0                           │
│                      derivatives                       │                                                        │
│     test/rank_1_of_NP_class/Diphenyl ethers (DPEs)     │                          0.0                           │
│          test/rank_1_of_NP_class/Diterpenoids         